In [384]:
from ctgan import CTGAN
from ctgan import load_demo
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import sample
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils import shuffle
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

In [ ]:
pd.options.display.max_columns = None
Ka = pd.read_excel("川崎症病人(訓練用 第一階段).xlsx")
fever = pd.read_excel('發燒病人(訓練用 第一階段).xlsx')
KD_val = pd.read_excel('KD_validation.xlsx')
FC_val = pd.read_excel('FC_validation.xlsx')
cmgh_20to23 = pd.read_excel('2020-23高長NewKD.xlsx')
Ka

In [ ]:
print('川崎症病人(原): ', len(Ka))
print('川崎症病人(<=5歲): ', len(Ka))
#發燒患者已只含0~6(不含6)歲
print('發燒病人: ', len(fever[fever['年齡(日)']<=1826]))
print('發燒病人: ', len(fever))


print('川崎症病人(驗證): ', len(KD_val))
print('發燒病人(驗證): ', len(FC_val))
print('川崎症病人(驗證): ', len(KD_val[KD_val['年齡(日)']<=1826]))
print('發燒病人(驗證): ', len(FC_val[FC_val['年齡(日)']<=1826]))
Ka = Ka[Ka['年齡(日)']<=1826]
fever = fever[fever['年齡(日)']<=1826]
KD_val = KD_val[KD_val['年齡(日)']<=1826]
FC_val = FC_val[FC_val['年齡(日)']<=1826]

In [387]:
cmgh_20to23['確診日期'] = pd.to_datetime(cmgh_20to23['確診日期'])
cmgh_20to23['BOD'] = pd.to_datetime(cmgh_20to23['BOD'])

cmgh_20to23['年齡(日)'] = (cmgh_20to23['確診日期']-cmgh_20to23['BOD']).dt.days

In [ ]:
print('川崎症病人(長庚驗證): ', len(cmgh_20to23[cmgh_20to23['年齡(日)']<=1826]))
# cmgh_20to23 = cmgh_20to23[cmgh_20to23['年齡(日)']<=1826]

In [389]:
#   _for_syn 為配合合成資料

#   日期轉月份
# Ka['輸入日期(月)'] = Ka['輸入日期'].dt.month
# fever['輸入日期(月)'] = fever['輸入日期'].dt.month
KD_val['輸入日期(月)'] = KD_val['輸入日期'].dt.month
FC_val['輸入日期(月)'] = FC_val['輸入日期'].dt.month
KD_val_for_syn = KD_val
FC_val_for_syn = FC_val
cmgh_20to23['輸入日期(月)'] = cmgh_20to23['確診日期'].dt.month
cmgh_20to23_for_syn = cmgh_20to23

#   one-hot
# ka_month = pd.get_dummies(Ka['輸入日期(月)'], prefix='Month')
# fever_month = pd.get_dummies(fever['輸入日期(月)'], prefix='Month')
KD_month_val = pd.get_dummies(KD_val['輸入日期(月)'], prefix='Month')
FC_month_val = pd.get_dummies(FC_val['輸入日期(月)'], prefix='Month')
val_month = pd.get_dummies(cmgh_20to23['輸入日期(月)'], prefix='Month')

#   檔案串接    raw + one-hot
# Ka = pd.concat([Ka, ka_month], axis=1)
# fever = pd.concat([fever, fever_month], axis=1)
KD_val = pd.concat([KD_val, KD_month_val], axis=1)
FC_val = pd.concat([FC_val, FC_month_val], axis=1)
cmgh_20to23 = pd.concat([cmgh_20to23, val_month], axis=1)

#   刪除多餘欄位
# Ka = Ka.drop(columns=['輸入日期(月)', '輸入日期'])  #'輸入日期(月)', 
# fever = fever.drop(columns=['輸入日期(月)', '輸入日期'])    #'輸入日期(月)', 
KD_val = KD_val.drop(columns=['輸入日期(月)', '輸入日期'])
FC_val = FC_val.drop(columns=['輸入日期(月)', '輸入日期'])
KD_val_for_syn = KD_val_for_syn.drop(columns=['輸入日期'])
FC_val_for_syn = FC_val_for_syn.drop(columns=['輸入日期'])
cmgh_20to23 = cmgh_20to23.drop(columns=['輸入日期(月)', '確診日期'])
cmgh_20to23_for_syn = cmgh_20to23_for_syn.drop(columns=['確診日期'])

In [390]:
cmgh_20to23.loc[cmgh_20to23['U/A WBC esterase']=='Positive', 'U/A WBC esterase'] = 1
cmgh_20to23.loc[cmgh_20to23['U/A WBC esterase']=='negative', 'U/A WBC esterase'] = 0
cmgh_20to23_for_syn.loc[cmgh_20to23_for_syn['U/A WBC esterase']=='Positive', 'U/A WBC esterase'] = 1
cmgh_20to23_for_syn.loc[cmgh_20to23_for_syn['U/A WBC esterase']=='negative', 'U/A WBC esterase'] = 0

#   刪除多餘欄位
cmgh_20to23 = cmgh_20to23.drop(columns=["KD","BOD","Chart No", "KD1 Age", "MCV (fL)","RDW-SD (fL)"])
cmgh_20to23_for_syn = cmgh_20to23_for_syn.drop(columns=["KD","BOD","Chart No", "KD1 Age", "MCV (fL)","RDW-SD (fL)"])

In [391]:
Ka = shuffle(Ka, random_state=30)
fever = shuffle(fever, random_state=30)
KD_val = shuffle(KD_val, random_state=30)
FC_val = shuffle(FC_val, random_state=30)
KD_val_for_syn = shuffle(KD_val_for_syn, random_state=30)
FC_val_for_syn = shuffle(FC_val_for_syn, random_state=30)

Ka['label'] = 1
fever['label'] = 0
KD_val['label'] = 1
FC_val['label'] = 0
KD_val_for_syn['label'] = 1
FC_val_for_syn['label'] = 0
cmgh_20to23['label'] = 1
cmgh_20to23_for_syn['label'] = 1

all_train_data = pd.concat([Ka, fever], ignore_index=True)
validation_data = pd.concat([KD_val, FC_val], ignore_index=True)
validation_data_for_syn = pd.concat([KD_val_for_syn, FC_val_for_syn], ignore_index=True)

In [392]:
x = all_train_data.values[:,:-1]
y = all_train_data.values[:,-1]
x_val = validation_data.values[:,:-1]
y_val = validation_data.values[:,-1]
x_val_for_syn = validation_data_for_syn.values[:,:-1]
y_val_for_syn = validation_data_for_syn.values[:,-1]
cmgh_20to23_x_test = cmgh_20to23.values[:,:-1]
cmgh_20to23_y_test = cmgh_20to23.values[:,-1]
cmgh_20to23_x_test_for_syn = cmgh_20to23_for_syn.values[:,:-1]
cmgh_20to23_y_test_for_syn = cmgh_20to23_for_syn.values[:,-1]

tmp_x_train, tmp_x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, stratify=y, random_state=30)
FC_test_index = [index for (index, x) in enumerate(y_test) if x==0]     #取出測試資料中FC的index
KD_test_index = [index for (index, x) in enumerate(y_test) if x==1]     #取出測試資料中KD的index
cmgh_20to23_y_test = cmgh_20to23_y_test.astype(float)
cmgh_20to23_y_test_for_syn = cmgh_20to23_y_test_for_syn.astype(float)
y_train = y_train.astype('float')
y_test = y_test.astype('float')

In [393]:
x_col = ['WBC','RBC','Hemoglobin','Hematocrit','MCH','MCHC','RDW','Platelets','Segment','Band','Lymphocyte','Monocyte','Eosinophil','Basophil','AST','ALT','CRP','UWBC','性別','年齡(日)','膿尿', '輸入日期']
y_col = ['label']
#   長庚 資料處理
#   分為合成訓練  原始訓練
df_tmp_x_train_for_syn = pd.DataFrame(tmp_x_train, columns=x_col)
df_tmp_x_test_for_syn = pd.DataFrame(tmp_x_test, columns=x_col)
df_tmp_x_train = pd.DataFrame(tmp_x_train, columns=x_col)
df_tmp_x_test = pd.DataFrame(tmp_x_test, columns=x_col)
df_tmp_y_train = pd.DataFrame(y_train, columns=y_col)

df_tmp_x_train_for_syn['輸入日期(月)'] = df_tmp_x_train_for_syn['輸入日期'].dt.month
df_tmp_x_test_for_syn['輸入日期(月)'] = df_tmp_x_test_for_syn['輸入日期'].dt.month
df_tmp_x_train['輸入日期(月)'] = df_tmp_x_train['輸入日期'].dt.month
df_tmp_x_test['輸入日期(月)'] = df_tmp_x_test['輸入日期'].dt.month

df_tmp_x_train_month = pd.get_dummies(df_tmp_x_train['輸入日期(月)'], prefix='Month')
df_tmp_x_test_month = pd.get_dummies(df_tmp_x_test['輸入日期(月)'], prefix='Month')

df_tmp_x_train = pd.concat([df_tmp_x_train, df_tmp_x_train_month], axis=1)
df_tmp_x_test = pd.concat([df_tmp_x_test, df_tmp_x_test_month], axis=1)

df_tmp_x_train_for_syn = df_tmp_x_train_for_syn.drop(columns=['輸入日期'])
df_tmp_x_test_for_syn = df_tmp_x_test_for_syn.drop(columns=['輸入日期'])
df_tmp_x_train = df_tmp_x_train.drop(columns=['輸入日期(月)', '輸入日期'])  #'輸入日期(月)', 
df_tmp_x_test = df_tmp_x_test.drop(columns=['輸入日期(月)', '輸入日期'])    #'輸入日期(月)', 

df_tmp_train_for_syn = pd.concat([df_tmp_x_train_for_syn, df_tmp_y_train], axis=1)

x_train = df_tmp_x_train.values
x_test = df_tmp_x_test.values
x_test_syn = df_tmp_x_test_for_syn.values

In [ ]:
df_tmp_train_for_syn_FC = df_tmp_train_for_syn[df_tmp_train_for_syn['label']==0]
df_tmp_train_for_syn_KD = df_tmp_train_for_syn[df_tmp_train_for_syn['label']==1]

df_tmp_train_for_syn_KD = df_tmp_train_for_syn_KD.drop('label',axis=1)

float_col = ['WBC','RBC','Hemoglobin','Hematocrit','MCH','MCHC','RDW','Platelets','Segment','Band','Lymphocyte','Monocyte','Eosinophil','Basophil','AST','ALT','CRP','UWBC']
int_col = ['性別','年齡(日)','膿尿']

for i in float_col:
    df_tmp_train_for_syn_KD[i] = df_tmp_train_for_syn_KD[i].astype('float64')
for i in int_col:
    df_tmp_train_for_syn_KD[i] = df_tmp_train_for_syn_KD[i].astype('int64')

#   建立 metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_tmp_train_for_syn_KD)

#   建立模型
synthesizer = CTGANSynthesizer(metadata)
synthesizer.fit(df_tmp_train_for_syn_KD)

In [395]:
# #   儲存模型
# synthesizer.save(
#     filepath='Sdv_CTGAN_0707_1.pkl'
# )

In [396]:
# #   執行儲存的模型
# synthesizer = CTGANSynthesizer.load(
#     filepath='Sdv_CTGAN_0707_1.pkl'
# )
# synthesizer.get_parameters()

In [ ]:
#   CTGAN 合成資料
synthesizer.reset_sampling()
print(len(df_tmp_train_for_syn_FC))
GAN_synthetic_data = synthesizer.sample(num_rows=int((len(df_tmp_train_for_syn_FC))/4)-len(df_tmp_train_for_syn_KD))
print(len(GAN_synthetic_data))

In [404]:
# GAN_synthetic_data.to_csv('GAN_synthetic_0817_3(1_4).csv', index=False)

In [399]:
# GAN_synthetic_data = pd.read_csv("GAN_synthetic_0707_2.csv")

In [400]:
GAN_synthetic_data['label'] = 1
df_tmp_train_for_syn_KD['label'] = 1

#   真實資料 + 合成資料
all_train_data_GAN = pd.concat([df_tmp_train_for_syn_FC, df_tmp_train_for_syn_KD, GAN_synthetic_data], ignore_index=True)
all_train_data_GAN = shuffle(all_train_data_GAN, random_state=30)

x_train_GAN = all_train_data_GAN.values[:,:-1]
y_train_GAN = all_train_data_GAN.values[:,-1]
y_train_GAN = y_train_GAN.astype('float')

In [401]:
#   驗證集整合

#   長庚raw + 高醫 + 長庚2020-22
# test_index = KD_test_index[32:133] + FC_test_index
# x_test = np.append(x_val, x_test[test_index], axis=0)
# y_test = np.append(y_val, y_test[test_index], axis=0)
# x_test = np.append(cmgh_20to23_x_test, x_test, axis=0)
# y_test = np.append(cmgh_20to23_y_test, y_test, axis=0)

# # x_test = np.append(x_val, x_test[sample(neg_index, 9000)], axis=0)
# # y_test = np.append(y_val, y_test[sample(neg_index, 9000)], axis=0)

#   使用純高醫資料驗證
# x_test = x_val
# y_test = y_val

#   針對訓練集中結合合成資料 驗證集整合     y_test按照先前即可
# test_index = KD_test_index[32:133] + FC_test_index
# x_test_syn = np.append(x_val_for_syn , x_test_syn[test_index], axis=0)
# x_test_syn = np.append(cmgh_20to23_x_test_for_syn, x_test_syn, axis=0)

#   高醫
# x_test_syn = x_val_for_syn


In [ ]:
xgb = XGBClassifier()   #scale_pos_weight=35
xgb.fit(x_train_GAN, y_train_GAN)
xg_y_pred_proba = xgb.predict_proba(x_test_syn)
xg_thresh_list = []
xg_accuracy_score_list = []
xg_precision_score_list = []
xg_recall_score_list = []
xg_f1_score_list = []
xg_record_f2_score = []
xg_specificity_score_list = []
xg_npv_score_list = []
xg_confusion_matrix_list = []
xg_y_pred_record = []
for th in range(1,101):
    th *= 0.01

    y_pred = [0 if (x[0] >= th) else 1 for x in xg_y_pred_proba]

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    npv = tn / (fn+tn)
    xg_thresh_list.append(th)
    xg_accuracy_score_list.append(accuracy_score(y_test, y_pred) * 100)
    xg_precision_score_list.append(precision_score(y_test, y_pred) * 100)
    xg_recall_score_list.append(recall_score(y_test, y_pred) * 100)
    xg_f1_score_list.append(f1_score(y_test, y_pred) * 100)
    xg_record_f2_score.append(fbeta_score(y_test, y_pred, beta=2) * 100)
    xg_specificity_score_list.append(specificity * 100)
    xg_npv_score_list.append(npv * 100)
    xg_confusion_matrix_list.append([tn, fp, fn, tp])
    xg_y_pred_record.append(y_pred)
recall_standard_list = [80, 85, 90, 95]
compare = []
for recall_standard in recall_standard_list:
    high_recall_f1score = []
    high_recall_f2score = []
    high_recall_accuracy = []
    high_recall_recall = []
    high_recall_precision = []
    high_recall_specificity = []
    high_recall_npv = []
    high_recall_confusion_matrix = []
    high_y_pred_record = []
    for index, recall in enumerate(xg_recall_score_list):
        if recall >= recall_standard:
            high_recall_f1score.append(xg_f1_score_list[index])
            high_recall_f2score.append(xg_record_f2_score[index])
            high_recall_accuracy.append(xg_accuracy_score_list[index])
            high_recall_precision.append(xg_precision_score_list[index])
            high_recall_recall.append(xg_recall_score_list[index])
            high_recall_specificity.append(xg_specificity_score_list[index])
            high_recall_npv.append(xg_npv_score_list[index])
            high_recall_confusion_matrix.append(xg_confusion_matrix_list[index])
            high_y_pred_record.append(xg_y_pred_record[index])
    high_recall_best_f2_score_index = np.argmax(high_recall_f2score)
    print("當Recall高於", recall_standard, "% 時")
    print("且選取最高的F2-score時")
    print("Recall為為:", high_recall_recall[high_recall_best_f2_score_index], "%")
    print("Specificity為:", high_recall_specificity[high_recall_best_f2_score_index], "%")
    print("Precision為:", high_recall_precision[high_recall_best_f2_score_index], "%")
    print("NPV為:", high_recall_npv[high_recall_best_f2_score_index], "%")
    print("+LR:", (high_recall_recall[high_recall_best_f2_score_index]/100)/(1-(high_recall_specificity[high_recall_best_f2_score_index]/100)))
    print("-LR:", (1-(high_recall_recall[high_recall_best_f2_score_index]/100))/(high_recall_specificity[high_recall_best_f2_score_index]/100))
    print("F1 Score為:", high_recall_f1score[high_recall_best_f2_score_index], "%")
    print("F2 Score為:", high_recall_f2score[high_recall_best_f2_score_index], "%")
    print("Accuracy為:", high_recall_accuracy[high_recall_best_f2_score_index], "%")
    print("TN, FP, FN, TP", high_recall_confusion_matrix[high_recall_best_f2_score_index])
    # print('預測值', high_y_pred_record[high_recall_best_f2_score_index])
    print(len(high_y_pred_record[high_recall_best_f2_score_index]))
    print("============================================")
    compare.append(high_y_pred_record[high_recall_best_f2_score_index])

In [ ]:
#pr curve
xg_y_pred_pr = [x[1] for x in xg_y_pred_proba]
x = np.arange(0, 1, 0.001)
y = np.arange(1, 0, -0.001)
fig, ax = plt.subplots()
# xg_precision, xg_recall, xg_thresholds = metrics.precision_recall_curve(y_test, xg_y_pred_pr, pos_label=1)
# plt.plot(xg_recall, xg_precision, color='m', label = "XG PR curve(val)")
PrecisionRecallDisplay.from_predictions(y_test, xg_y_pred_pr, name="XG PR curve", ax=ax)
# precision, recall, thresholds = metrics.precision_recall_curve(y_test, i, pos_label=1)
# plt.plot(recall, precision, color='y', label = "ENS PR curve(val)")
# PrecisionRecallDisplay.from_predictions(y_test, i, name="ENS PR curve", ax=ax)
# plt.plot(x, y, color='0', linestyle = "-.")

plt.ylabel("Precision")
plt.xlabel("Recall")
plt.legend(loc=3)
plt.grid(True)
# plt.savefig("0to5_pr_curve_ENS(val_pure_kao).jpg", format = "jpg", dpi=500)
plt.show()